# ERP Structured Ingestion
Normalize Orders, SKUs, Customers and write curated tables.

In [ ]:
import pandas as pd
from datetime import datetime

# Example: read landing CSVs (replace with OneLake paths in Fabric)
orders = pd.read_csv('landing/erp/orders.csv')
skus = pd.read_csv('landing/erp/skus.csv')
customers = pd.read_csv('landing/erp/customers.csv')

# Clean and standardize
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
skus['SKU'] = skus['SKU'].astype(str).str.strip().str.upper()

# Star schema example
dim_sku = skus[['SKU','Description','Weight','Length','Width','Height']].drop_duplicates()
dim_customer = customers[['CustomerId','Name','Region']].drop_duplicates()
fact_shipment = orders.rename(columns={'CustomerId':'CustomerId'})

# Persist (replace with lakehouse Delta writes in Fabric)
dim_sku.to_parquet('curated/DimSKU.parquet', index=False)
dim_customer.to_parquet('curated/DimCustomer.parquet', index=False)
fact_shipment.to_parquet('curated/FactShipment.parquet', index=False)
print('Wrote curated ERP tables at', datetime.utcnow(), 'UTC')
